# Get gender

## Add `'gender'` key to database

In [1]:
import requests
from IPython.display import JSON
from gender_guesser_first_names import gender
from re import sub
import json

In [2]:
LOCAL=False
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [3]:
D=gender.Detector(case_sensitive=False)

In [6]:
def force_get_gender(d):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    gender = 'unknown'
    names = ''
    if (d.get('name') and d.get('name').get('value') and 
        sub('\w\.','',d.get('name').get('value').split(', ')[-1]).strip()):
        names = sub('\w\.','',d.get('name').get('value').split(', ')[-1]).strip()
    if not names and d.get('full_name'):
        names = sub('\w\.','',d.get('full_name').split(', ')[-1]).strip()
    if names:
        if d.get('country'):
            gender = D.get_gender( names, country=d.get('country').strip() )
        else:
            gender = D.get_gender( names )
    return gender

In [7]:
kk = [d.update( {'gender': force_get_gender(d)} ) for d in df ]

In [8]:
f=open('inspire_LA_gender.json','w')
json.dump(df,f)
f.close()

## Analysis

In [9]:
import pandas as pd

In [10]:
DF = pd.read_json('inspire_LA_gender.json')

Unique authors ids

In [11]:
DF.shape[0]

57588

In [12]:
DF = DF.drop_duplicates('author_id').reset_index(drop=True)
DF.shape[0]

29969

In [13]:
DF['first_names'] = DF['name'].str.get('value').astype(str).str.split(', ').str[-1]#.apply(
    #lambda s: sub('\w\.','',s)).str.replace('None','')

In [14]:
DF['gender'].unique()

array(['male', 'unknown', 'female', 'andy'], dtype=object)

In [15]:
print(DF['gender'].shape[0])
total = DF['gender'].dropna().shape[0]
total

29969


29969

In [16]:
male = DF[DF['gender'] == 'male'].shape[0]
female = DF[DF['gender'] == 'female'].shape[0]
unknown = DF[DF['gender'] == 'unknown'].shape[0]
unknown = unknown + DF[DF['gender'] == 'andy'].shape[0]

In [ ]:
assert male + female + unknown == total

In [20]:
male+female

17335

In [17]:
male

14626

In [18]:
female

2709

In [21]:
round(female/male,2)

0.19

In [22]:
unknown

12634

In [23]:
DFFM = DF[ (( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFFM.shape

(17335, 14)

In [24]:
DFU = DF[ ~(( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFU.shape

(12634, 14)

In [25]:
DFFM[['first_names','country','gender']].sample(50)

,first_names,country,gender
10346,Andrey A.,Argentina,male
25449,Di,United States,female
22457,Gabor Gyula,Hungary,male
26525,Boris,Israel,male
26491,Gernot,Germany,male
29575,None,United States,female
2171,Hector,Costa Rica,male
28007,Jana B.,Portugal,female
9375,Richard S.,United States,male
12582,None,Portugal,male


In [26]:
#[['first_names','country','gender']]
DFU.sample(50)[['author_id','full_name','name','country','gender']]

,author_id,full_name,name,country,gender
20038,D.D.Maccarrone.1,"Maccarrone, G.D.",None,Italy,unknown
2818,C.N.Cruz.Camacho.1,"Cruz-Camacho, C.N.",None,Colombia,unknown
1787,A.Altamirano.1,"Altamirano, A.",None,Peru,unknown
17382,D.C.Rivera.Agudelo.2,"Rivera-Agudelo, D.C.",None,Mexico,unknown
13122,L.Z.Zhang.1,"Zhang, Long-Zhe",None,United States,unknown
26390,R.Dos.Santos.1,"dos Santos, Rubia",None,Brazil,unknown
26224,M.F.Nieva.1,"Nieva, M.F.",None,Brazil,unknown
6673,O.Herrera.1,"Herrera, O.",None,Chile,unknown
21587,K.L.Ribeiro.1,"Ribeiro, K.L.",None,Brazil,unknown
18462,A.Z.Goldberg.1,"Goldberg, A.Z.","{'value': 'Goldberg, A.Z.'}",Canada,unknown
